import packages

In [1]:
# import sys, os
# from pathlib import Path
# sys.path.append(str(Path(os.getcwd()))+'/..')

import os
os.chdir('..')

from analysis.fidelity_prediction import FidelityModel
from baselines.fidelity_prediction.rb_prediction import RBModel
from simulator.gate_error_model import GateErrorModel

from analysis.vectorization import RandomwalkModel

from data_objects.random_circuit import random_circuits, random_circuit
from data_objects.backend import Backend, LinearBackend, GridBackend, FullyConnectedBackend

from simulator.noisy_simulator import NoisySimulator
import random

from tools.ray_func import map


model settings

In [2]:
n_qubits = 18
n_steps = 2
n_walks = 20
backend = LinearBackend(n_qubits, 1)


generate random circuit to train

In [ ]:
circuits = random_circuits(backend, n_circuits=500, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

train upstream model, turn a circuit to vectors using random walk

In [ ]:
vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
gate_vecs_per_circuit = vec_model.train(circuits, multi_process=False, remove_redundancy = False)

start random walk for 501 circuits


100%|██████████| 501/501 [00:06<00:00, 78.08it/s] 


count path
device size after random walk = 9
0's path table size = 75
1's path table size = 150
2's path table size = 197
3's path table size = 150
4's path table size = 75
(0, 1)'s path table size = 95
(1, 2)'s path table size = 146
(2, 3)'s path table size = 146
(3, 4)'s path table size = 95


select interaction patterns randomly, simulate interaction between gates

In [ ]:
all_paths = vec_model.all_paths()
high_error_paths = random.choices(all_paths, k = 20)

error_model = GateErrorModel.random_model(backend=backend, high_error_paths=high_error_paths)
error_model.vec_model = vec_model

simulator = NoisySimulator(backend=backend, gate_error_model = error_model)
ground_truth_fidelities = map(lambda circuit: simulator.obtain_circuit_fidelity(circuit)[0], circuits, show_progress=False, multi_process=True)

train fidelity prediction model

In [ ]:
fidelity_model = FidelityModel(vec_model)
fidelity_model.train((circuits, ground_truth_fidelities))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


len(train dataset) = 400, len(validation dataset) = 101
epoch: 0, 	 epoch loss = 3770.66259765625, 	 validation loss = 6.224177837371826
epoch: 100, 	 epoch loss = 773.9028930664062, 	 validation loss = 1.8807038068771362
epoch: 200, 	 epoch loss = 459.62591552734375, 	 validation loss = 1.3457980155944824
epoch: 300, 	 epoch loss = 387.0428771972656, 	 validation loss = 1.2294397354125977
epoch: 400, 	 epoch loss = 344.41131591796875, 	 validation loss = 1.1326135396957397
epoch: 500, 	 epoch loss = 307.116455078125, 	 validation loss = 1.0325236320495605
epoch: 600, 	 epoch loss = 276.13623046875, 	 validation loss = 0.9459999799728394
epoch: 700, 	 epoch loss = 251.72381591796875, 	 validation loss = 0.876919150352478
epoch: 800, 	 epoch loss = 231.47145080566406, 	 validation loss = 0.8180903792381287
epoch: 900, 	 epoch loss = 213.94674682617188, 	 validation loss = 0.766452431678772
finish taining


predict on test dataset

In [ ]:
test_circuits = random_circuits(backend, n_circuits=50, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)
test_ground_truth_fidelities = map(lambda circuit: simulator.obtain_circuit_fidelity(circuit)[0], test_circuits, show_progress=False, multi_process=True)

test_predicts= []
for test_cir in test_circuits:
    test_predicts.append(fidelity_model.predict_circuit_fidelity(test_cir))
print(test_ground_truth_fidelities)
print(test_predicts)

[0.7051999999999999, 0.6843999999999999, 0.5862, 0.3728, 0.7407999999999999, 0.8042, 0.8022000000000002, 0.7734, 0.6442000000000001, 0.8866000000000002, 0.5471999999999999, 0.5396, 0.6237999999999999, 0.651, 0.7642, 0.7382000000000001, 0.6554, 0.6900000000000001, 0.4633999999999999, 0.7328, 0.4002, 0.788, 0.4072, 0.43079999999999996, 0.4842000000000001, 0.6610000000000001, 0.24880000000000005, 0.5038, 0.32520000000000004, 0.30400000000000005, 0.4946, 0.4915999999999999, 0.45499999999999996, 0.6628000000000001, 0.41679999999999995, 0.19600000000000004, 0.21939999999999998, 0.14480000000000004, 0.17100000000000004, 0.1516000000000001, 0.3412, 0.14040000000000002, 0.4497999999999999, 0.2172, 0.4299999999999999, 0.26860000000000006, 0.19960000000000006, 0.4567999999999999, 0.4016, 0.21260000000000004, 0.18080000000000016]
[Array(0.6608869, dtype=float32), Array(0.6247604, dtype=float32), Array(0.64236873, dtype=float32), Array(0.5618794, dtype=float32), Array(0.69217706, dtype=float32), Ar

compare with RB predict model

In [ ]:
rb_modle = RBModel(simulator)
test_rb_predicts= []
for test_cir in test_circuits:
    test_rb_predicts.append(rb_modle.get_rb_fidelity(test_cir))

print(test_rb_predicts)

[0.9087011626351813, 0.7758030965384289, 0.7994830480958711, 0.7396075453719695, 0.7760607627365118, 0.9228510740484697, 0.8145671854721591, 0.8864710275682284, 0.8949891937923592, 0.7934994819209237, 0.7798037879718112, 0.8733277611136399, 0.8646689677866424, 0.8293431239317204, 0.7705280169632022, 0.7958722152492719, 0.8010201752113163, 0.8216787065044853, 0.7111595761473025, 0.7518635604183198, 0.7306439263009161, 0.7723446561574304, 0.6241369095928204, 0.6865435004822139, 0.7746077733195673, 0.8328771008955906, 0.6737689776183271, 0.7878174144969367, 0.6709338668291787, 0.7264632101357454, 0.7234063763258862, 0.7922968568781775, 0.6018155325587142, 0.7239650978371289, 0.5803222140778819, 0.4000097634957905, 0.5643767640877658, 0.39121250228205084, 0.4938635604195252, 0.4238845649025567, 0.42218357963209396, 0.5443475425503982, 0.588601100525215, 0.48485635223191853, 0.6274526132863091, 0.49461254007053207, 0.5051832247144242, 0.5792808141973335, 0.5218066090145087, 0.60924900645405

export excel

In [ ]:
# TODO: on real world quantum hardware
all_path_errors = []


all_path_errors = fidelity_model.get_all_path_errors()
from openpyxl import Workbook

wb = Workbook()
ws = wb.active

ws['A1'] = 'Path'
ws['B1'] = 'error'
ws['C1'] = 'in noise model'

for path,error in all_path_errors:
    ws.append((path , error, path in high_error_paths))

wb.save('all_path_errors.xlsx')
